In [ ]:
import os, pickle, sqlite3
import matplotlib.pyplot as plt
from tkinter.filedialog import askopenfilename
#
filename = askopenfilename()

In [ ]:
directory = os.path.dirname(filename)
with open(filename, 'rb') as f:
    data = pickle.load(f)
# for now:
savefigs = False

In [ ]:
timestamps = sorted(list(data.keys()))
charges = {}
pulses = {}
ibis = {}
for x in timestamps:
    if data[x].startswith(b'S'):
        charges[x] = data[x]
    elif data[x].startswith(b'B'):
        pulses[x] = data[x]
    elif data[x].startswith(b'Q'):
        ibis[x] = data[x]
    else:
        print('Unit of unknown type found')

In [ ]:
db = sqlite3.Connection('../db.sqlite')
curs = db.cursor()
tables = curs.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;").fetchall()[2:]
tables = [x[0] for x in tables]
for x in tables:
    start_time = curs.execute('SELECT MIN(Timestamp) FROM ' + x + ';').fetchall()[0][0]
    if start_time > timestamps[0] and start_time < timestamps[-1]:
        table = x
        break
#
try:
    print('Using table: ' + table)
except NameError:
    print('Could not find a table with timestamps matching EEG data.')
    table = None
    curs.close()
    db.close()
#
if table is not None:
    corrects = curs.execute('select timestamp from ' + table + ' where Correct=1;').fetchall()
    incorrects = curs.execute('select timestamp from ' + table + ' where Correct=0;').fetchall()
#
os.chdir(directory)

In [ ]:
#width = len(timestamps) // 25
#height = width // 5

In [ ]:
charge_times = sorted(list(charges.keys()))
charge_values = [charges[x] for x in charge_times]
#
width = len(charge_times) // 25
height = width // 5
#
plt.figure(figsize=(width,height))
plt.plot(charge_times, charge_values, color='black')
plt.xlabel('Time')
plt.ylabel('Charge')
if table is not None:
    for a in corrects:
        plt.axvline(x=a, color='purple')
    for b in incorrects:
        plt.axvline(x=b, color='orange')
plt.show()
if savefigs:
    plt.savefig('ekg_charge')

In [ ]:
pulse_times = sorted(list(pulses.keys()))
pulse_values = [pulses[x] for x in pulse_times]
#
width = len(pulse_times) // 25
height = width // 5
#
plt.figure(figsize=(width,height))
plt.plot(pulse_times, pulse_values, color='black')
plt.xlabel('Time')
plt.ylabel('Pulse Rate')
if table is not None:
    for a in corrects:
        plt.axvline(x=a, color='purple')
    for b in incorrects:
        plt.axvline(x=b, color='orange')
plt.show()
if savefigs:
    plt.save('ekg_pulse')

In [ ]:
ibi_times = sorted(list(ibis.keys()))
ibi_values = [ibis[x] for x in ibi_times]
#
width = len(ibi_times) // 25
height = width // 5
#
plt.figure(figsize=(width,height))
plt.plot(ibi_values, ibi_times, color='black')
plt.xlabel('Time')
plt.ylabel('Inter-Beat Interval?')
if table is not None:
    for a in corrects:
        plt.axvline(x=a, color='purple')
    for b in incorrects:
        plt.axvline(x=b, color='orange')
plt.show()
if savefigs:
    plt.save('ekg_ibi')

In [ ]:
try:
    curs.close()
    db.close()
except Exception:
    pass